<a href="https://colab.research.google.com/github/JoaoMMS98/ML_2.0/blob/main/Model_Selection_baseline_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Model Selection

In this notebook, we are going to apply different ways to split your dataset and try to select the best model.

__`Step 1`__ Import the needed libraries

In [1]:
!pip install xgboost==1.7.5
!pip install scikit-learn==1.2.2
!pip install catboost==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.5 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.3
    Uninstalling xgboost-2.1.3:
      Successfully uninstalled xgboost-2.1.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 20.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Chiel > XGBoost,CatBoost.

Mariana > KNN, RFM, SVM

In [16]:
# Import the data
train_df = pd.read_csv('traindf.csv')
val_df = pd.read_csv('valdf.csv')
test_df = pd.read_csv('testdf.csv')

In [17]:
train_df

,Claim Injury Type,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,First Hearing Date,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Agreement Reached,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,c2_to_c3_delta,assembly_delta,process_efficiency,complexity_time,processing_speed_rank,covid_risk_score,geo_risk_density,injury_complexity,ime_severity,severity_score,industry_wage_percentile,wage_to_industry_ratio,high_wage_risk
0,4. TEMPORARY,0.518093,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,PROPERTY AND CASUALTY,0.983139,QUEENS,NYC,-0.037533,0.920180,-1.101128,1.021693,0.703471,-0.735036,1.618148,11432.0,0.0,1.497677,0.092321,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.885803,-0.067728,-0.264434,1.532325,0.981248,-0.616113,1.504723,-0.501637,0.920180,0.265372,0.445502,1.307827,1.416506
1,3. MED ONLY,-0.875110,-0.908646,1.344678,0.955044,-0.783603,0.558209,HEALTH & HOSPITAL CORP.,-0.780636,BRONX,NYC,-1.477658,-0.445038,1.514005,1.021693,0.185530,-1.267597,1.018289,10451.0,0.0,-0.501860,0.632632,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.103642,-0.910028,0.114678,-0.475080,-0.410310,-0.688142,0.653292,-0.798647,-0.445038,-0.875899,1.312875,1.441940,1.416506
2,2. NON-COMP,0.078134,1.012027,-0.755254,-0.133220,0.864313,0.558209,AMERICAN ZURICH INSURANCE CO,0.983139,KINGS,NYC,-0.037533,-0.445038,-0.933566,1.021693,-1.277371,1.332596,-0.407201,11203.0,0.0,-0.001976,-0.447990,1.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.341702,1.012061,-0.175885,-0.475080,-0.410310,-0.779453,1.252910,-0.502181,-0.445038,-0.663056,-0.595344,-0.702162,-0.705963
3,2. NON-COMP,-1.315068,0.056452,-0.755254,1.317798,0.041721,0.558209,"NORDSTROM, INC.",-1.524399,KINGS,NYC,-0.037533,-0.445038,-0.260367,1.021693,-0.701976,0.572261,-1.077143,11237.0,0.0,-1.001744,0.092321,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.379772,0.055767,-0.090487,-0.475080,-0.410310,-0.272904,1.252910,-0.404400,-0.445038,-0.592856,-0.595344,-0.702162,-0.705963
4,3. MED ONLY,-0.068519,-0.788009,0.889468,0.156984,-0.698884,-1.896900,NEW YORK BLACK CAR OPERATORS',-1.524399,NASSAU,NYC,-1.413414,-0.445038,-0.417166,1.021693,-0.256659,-0.390479,1.018289,11003.0,0.0,0.497909,0.632632,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-0.801911,-0.789699,-0.141592,-0.475080,-0.410310,-0.043594,0.903836,-0.268457,-0.445038,-0.495259,-0.595344,-0.702162,-0.705963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342031,2. NON-COMP,-0.728457,-0.426097,-0.755254,0.737391,-0.373674,-2.035080,SARATOGA COUNTY WC PLAN,-0.780636,SARATOGA,ALBANY,-0.037533,-0.445038,0.958382,-0.440295,-1.275286,-0.735036,-0.699292,12803.0,0.0,-0.001976,0.092321,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.190212,-0.425547,-0.351311,-0.475080,-0.410310,-0.753699,-0.875856,-0.780720,-0.445038,-0.863029,-0.595344,-0.702162,-0.705963
342032,5. PPD SCH LOSS,1.178031,1.634261,1.251288,-1.221484,1.399953,0.490726,PENNSYLVANIA MANUFACTURERS',0.983139,MONROE,ROCHESTER,-0.037533,-0.445038,1.514005,-1.008469,-0.949807,1.332596,0.756937,14616.0,0.0,1.497677,-0.447990,1.0,1.0,1.0,0.0,0.0,0.0,0.0,-0.861834,1.632703,-0.261134,-0.475080,-0.410310,-0.246737,-0.368297,1.562490,-0.445038,0.819237,0.532239,1.346586,1.416506
342033,3. MED ONLY,1.617990,-1.321353,-0.755254,-1.511687,-1.144341,0.558209,MORAVIA CENTRAL SCHOOL DIST,-0.780636,CAYUGA,SYRACUSE,-0.037533,-0.445038,-0.244270,-0.440295,0.703471,-0.390479,1.018289,13118.0,0.0,-0.001976,0.632632,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.420035,-1.321677,0.040137,-0.475080,-0.410310,-0.366419,-0.995450,-0.268457,-0.445038,-0.495259,-0.595344,-0.702162,-0.705963
342034,2. NON-COMP,-0.215172,0.065976,-0.755254,0.229535,0.049920,0.558209,ARCH INDEMNITY INSURANCE CO.,0.983139,NASSAU,NYC,

In [14]:
pd.set_option('display.max_columns', None)
train_df

,Claim Injury Type,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,First Hearing Date,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,c2_to_c3_delta,assembly_delta,process_efficiency,complexity_time,processing_speed_rank,covid_risk_score,geo_risk_density,injury_complexity,ime_severity,severity_score,industry_wage_percentile,wage_to_industry_ratio,high_wage_risk
0,4. TEMPORARY,0.518093,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,PROPERTY AND CASUALTY,0.983139,QUEENS,NYC,-0.039361,0.920180,-1.101128,1.021693,0.703471,-0.735036,1.618148,1.497677,0.092356,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.885803,-0.067728,-0.264434,1.532325,0.981248,-0.616113,1.504723,-0.501637,0.920180,0.265372,0.445502,1.307827,1.416506
1,3. MED ONLY,-0.875110,-0.908646,1.344678,0.955044,-0.783603,0.558209,HEALTH & HOSPITAL CORP.,-0.780636,BRONX,NYC,-1.479688,-0.445038,1.514005,1.021693,0.185530,-1.267597,1.018289,-0.501860,0.634795,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.103642,-0.910028,0.114678,-0.475080,-0.410310,-0.688142,0.653292,-0.798647,-0.445038,-0.875899,1.312875,1.441940,1.416506
2,2. NON-COMP,0.078134,1.012027,-0.755254,-0.133220,0.864313,0.558209,AMERICAN ZURICH INSURANCE CO,0.983139,KINGS,NYC,-0.039361,-0.445038,-0.933566,1.021693,-1.277371,1.332596,-0.407201,-0.001976,-0.450083,1.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.341702,1.012061,-0.175885,-0.475080,-0.410310,-0.779453,1.252910,-0.502181,-0.445038,-0.663056,-0.595344,-0.702162,-0.705963
3,2. NON-COMP,-1.315068,0.056452,-0.755254,1.317798,0.041721,0.558209,"NORDSTROM, INC.",-1.524399,KINGS,NYC,-0.039361,-0.445038,-0.260367,1.021693,-0.701976,0.572261,-1.077143,-1.001744,0.092356,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.379772,0.055767,-0.090487,-0.475080,-0.410310,-0.272904,1.252910,-0.404400,-0.445038,-0.592856,-0.595344,-0.702162,-0.705963
4,3. MED ONLY,-0.068519,-0.788009,0.889468,0.156984,-0.698884,-1.896900,NEW YORK BLACK CAR OPERATORS',-1.524399,NASSAU,NYC,-1.415435,-0.445038,-0.417166,1.021693,-0.256659,-0.390479,1.018289,0.497909,0.634795,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-0.801911,-0.789699,-0.141592,-0.475080,-0.410310,-0.043594,0.903836,-0.268457,-0.445038,-0.495259,-0.595344,-0.702162,-0.705963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342031,2. NON-COMP,-0.728457,-0.426097,-0.755254,0.737391,-0.373674,-2.035080,SARATOGA COUNTY WC PLAN,-0.780636,SARATOGA,ALBANY,-0.039361,-0.445038,0.958382,-0.440295,-1.275286,-0.735036,-0.699292,-0.001976,0.092356,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.190212,-0.425547,-0.351311,-0.475080,-0.410310,-0.753699,-0.875856,-0.780720,-0.445038,-0.863029,-0.595344,-0.702162,-0.705963
342032,5. PPD SCH LOSS,1.178031,1.634261,1.251288,-1.221484,1.399953,0.490726,PENNSYLVANIA MANUFACTURERS',0.983139,MONROE,ROCHESTER,-0.039361,-0.445038,1.514005,-1.008469,-0.949807,1.332596,0.756937,1.497677,-0.450083,1.0,1.0,1.0,0.0,0.0,0.0,0.0,-0.861834,1.632703,-0.261134,-0.475080,-0.410310,-0.246737,-0.368297,1.562490,-0.445038,0.819237,0.532239,1.346586,1.416506
342033,3. MED ONLY,1.617990,-1.321353,-0.755254,-1.511687,-1.144341,0.558209,MORAVIA CENTRAL SCHOOL DIST,-0.780636,CAYUGA,SYRACUSE,-0.039361,-0.445038,-0.244270,-0.440295,0.703471,-0.390479,1.018289,-0.001976,0.634795,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.420035,-1.321677,0.040137,-0.475080,-0.410310,-0.366419,-0.995450,-0.268457,-0.445038,-0.495259,-0.595344,-0.702162,-0.705963
342034,2. NON-COMP,-0.215172,0.065976,-0.755254,0.229535,0.049920,0.558209,ARCH INDEMNITY INSURANCE CO.,0.983139,NASSAU,NYC,-0.039361,-0.445038,1.514005,1.021693,0.893373,1.332596,-0.844992,1.497677,0.092356,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.372581,0.065266,-

In [5]:
train_df = train_df.drop(['Agreement Reached', 'Alternative Dispute Resolution_U', 'Zip Code'], axis=1, errors='ignore')
val_df = val_df.drop(['Agreement Reached', 'Alternative Dispute Resolution_U', 'Zip Code'], axis=1, errors='ignore')

In [18]:
# Get the column names of both DataFrames
train_cols = set(train_df.columns)
val_cols = set(val_df.columns)

# Find the differences
diff_train = train_cols - val_cols  # Columns in train_df but not in val_df
diff_val = val_cols - train_cols  # Columns in val_df but not in train_df

# Print the differences
if diff_train:
    print("Columns in train_df but not in val_df:")
    print(diff_train)

if diff_val:
    print("\nColumns in val_df but not in train_df:")
    print(diff_val)

if not diff_train and not diff_val:
    print("Both DataFrames have the same columns.")

Both DataFrames have the same columns.


In [19]:
categorical_cols = train_df.select_dtypes(include=['object']).columns
num_categorical_cols = len(categorical_cols)
print(f"Number of categorical columns: {num_categorical_cols}")
print(f"Categorical columns: {categorical_cols}")

Number of categorical columns: 4
Categorical columns: Index(['Claim Injury Type', 'Carrier Name', 'County of Injury',
       'District Name'],
      dtype='object')


## 1.1. The train-test split

8:2 splitting target column

In [31]:
numerical_features = train_df.select_dtypes(include=np.number).columns
categorical_features = train_df.select_dtypes(include=['object']).columns

for feature in numerical_features:
    median_value = train_df[feature].median()
    train_df[feature].fillna(median_value, inplace=True)

for feature in categorical_features:
    mode_value = train_df[feature].mode()[0]  # Get the most frequent value
    train_df[feature].fillna(mode_value, inplace=True)

In [32]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Prepare training data
X_train = train_df.drop('Claim Injury Type', axis=1)  # Independent variables
y_train = train_df['Claim Injury Type']  # Target variable

# Prepare validation/test data
X_test = val_df.drop('Claim Injury Type', axis=1)  # Independent variables
y_test = val_df['Claim Injury Type']  # Target variable

# Label encode the target variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Print the split ratio
total_samples = len(train_df) + len(val_df)
print('train:{}% | test:{}%'.format(
    round(len(train_df)/total_samples * 100, 2),
    round(len(val_df)/total_samples * 100, 2)
))

# Optional: Display all columns
pd.set_option('display.max_columns', None)

# Verify shapes
print("\nData shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

train:74.87% | test:25.13%

Data shapes:
X_train shape: (342036, 41)
X_test shape: (114805, 41)
y_train shape: (342036,)
y_test shape: (114805,)


In [ ]:
X_train

,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,First Hearing Date,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M
0,0.518093,-0.020370,1.353821,-0.495974,-0.028681,-2.352697,PROPERTY AND CASUALTY,0.497548,QUEENS,NYC,-0.037533,1.098612,0.017597,0.463375,0.051126,0.063035,0.090122,1.497677,0.092321,1,1,1,1,0,0,0
1,-0.875110,-0.511111,1.402126,0.955044,-0.531549,0.000000,HEALTH & HOSPITAL CORP.,0.212179,BRONX,NYC,-1.477658,0.000000,0.199081,0.463375,0.039230,0.010374,0.073124,-0.501860,0.632632,1,1,1,0,0,0,0
2,0.078134,0.609259,0.000000,-0.133220,0.621415,0.000000,AMERICAN ZURICH INSURANCE CO,0.497548,KINGS,NYC,-0.037533,0.000000,0.029226,0.463375,0.005629,0.267488,0.032729,-0.001976,-0.447990,1,1,1,0,0,0,1
3,-1.315068,0.051852,0.000000,1.317798,0.045889,0.000000,"NORDSTROM, INC.",0.091843,KINGS,NYC,-0.037533,0.000000,0.075944,0.463375,0.018845,0.192304,0.013745,-1.001744,0.092321,1,1,1,0,0,0,1
4,-0.068519,-0.440741,1.223679,0.156984,-0.472275,-3.170124,NEW YORK BLACK CAR OPERATORS',0.091843,NASSAU,NYC,-1.413414,0.000000,0.065062,0.463375,0.029073,0.097106,0.073124,0.497909,0.632632,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,-1.241742,-0.150000,1.409856,1.245247,-0.162524,-2.651452,INDEMNITY INSURANCE CO OF,0.497548,WESTCHESTER,NYC,0.396110,0.693147,0.199081,0.093731,0.007646,0.004915,0.014207,1.497677,0.092321,1,1,1,1,0,0,1
459216,0.884725,-0.009259,1.383011,-0.858729,-0.017208,-2.307054,REDWOOD FIRE & CASUALTY,0.497548,QUEENS,NYC,0.171258,1.945910,0.199081,0.463375,0.064603,0.192304,0.039561,-1.001744,0.092321,1,1,1,1,0,0,1
459217,1.031378,-0.292593,1.361083,-1.003831,-0.309751,0.000000,UTICA MUTUAL INS CO,0.497548,ERIE,BUFFALO,-0.037533,0.000000,0.023039,0.236296,0.020729,0.267488,0.083779,0.997793,0.092321,1,1,1,0,0,0,0
459218,-1.241742,0.642593,0.000000,1.172696,0.655832,0.000000,NEW HAMPSHIRE INSURANCE CO,0.497548,KINGS,NYC,-0.037533,0.000000,0.053783,0.463375,0.055488,0.267488,0.032729,-1.001744,-0.447990,1,1,1,0,0,0,1


Stratified Sampling: Ensures that the training and testing sets maintain the same class distribution as the original dataset, reducing bias.


Repeated Hold-Out: The process is repeated multiple times with different splits, and the average performance is considered.

In [33]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Identify categorical columns
categorical_features_indices = [i for i, dtype in enumerate(X_train.dtypes)
                              if dtype == 'object' or dtype == 'category']

# Create CatBoost Pools with categorical features
train_pool = Pool(data=X_train,
                 label=y_train,
                 cat_features=categorical_features_indices)

test_pool = Pool(data=X_test,
                label=y_test,
                cat_features=categorical_features_indices)

# Initialize model
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    verbose=0
)

# Fit using pools
model.fit(train_pool)

# Make predictions for both train and test sets
y_pred_train = model.predict(train_pool)
y_pred_test = model.predict(test_pool)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print(f'\nNumber of categorical features detected: {len(categorical_features_indices)}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

# Calculate performance difference between train and test
print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

Training Metrics:
Accuracy Score: 0.7998
Macro F1 Score: 0.3764

Testing Metrics:
Accuracy Score: 0.7979
Macro F1 Score: 0.3684

Number of categorical features detected: 3

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.79      0.32      0.45      2495
           1       0.85      0.99      0.91     58216
           2       0.46      0.06      0.11     13781
           3       0.75      0.91      0.82     29701
           4       0.70      0.58      0.63      9656
           5       0.00      0.00      0.00       842
           6       0.00      0.00      0.00        20
           7       1.00      0.01      0.02        94

    accuracy                           0.80    114805
   macro avg       0.57      0.36      0.37    114805
weighted avg       0.75      0.80      0.75    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): 0.0019
F1 Score Difference (Train - Test): 0.0080


In [34]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Save target variable before dropping categoricals
y_train = train_df['Claim Injury Type']  # Target variable
y_test = val_df['Claim Injury Type']  # Target variable

# Function to remove categorical columns
def drop_categorical_columns(df):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    return df.drop(columns=categorical_cols)

# Drop categorical columns from train and validation sets
X_train = drop_categorical_columns(train_df)
X_test = drop_categorical_columns(val_df)

# Label encode the target variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Print the split ratio
total_samples = len(train_df) + len(val_df)
print('train:{}% | test:{}%'.format(
    round(len(train_df)/total_samples * 100, 2),
    round(len(val_df)/total_samples * 100, 2)
))

# Optional: Display all columns
pd.set_option('display.max_columns', None)

# Verify shapes and remaining features
print("\nData shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Print information about removed features
original_features = train_df.shape[1] - 1  # Subtract 1 for target column
remaining_features = X_train.shape[1]
print(f"\nFeature reduction:")
print(f"Original number of features: {original_features}")
print(f"Remaining numeric features: {remaining_features}")
print(f"Number of categorical features removed: {original_features - remaining_features}")

train:74.87% | test:25.13%

Data shapes:
X_train shape: (342036, 38)
X_test shape: (114805, 38)
y_train shape: (342036,)
y_test shape: (114805,)

Feature reduction:
Original number of features: 41
Remaining numeric features: 38
Number of categorical features removed: 3


In [ ]:
X_train

,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Type,First Hearing Date,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M
0,0.518093,-0.020370,1.353821,-0.495974,-0.028681,-2.352697,0.497548,-0.037533,1.098612,0.017597,0.463375,0.051126,0.063035,0.090122,1.497677,0.092321,1,1,1,1,0,0,0
1,-0.875110,-0.511111,1.402126,0.955044,-0.531549,0.000000,0.212179,-1.477658,0.000000,0.199081,0.463375,0.039230,0.010374,0.073124,-0.501860,0.632632,1,1,1,0,0,0,0
2,0.078134,0.609259,0.000000,-0.133220,0.621415,0.000000,0.497548,-0.037533,0.000000,0.029226,0.463375,0.005629,0.267488,0.032729,-0.001976,-0.447990,1,1,1,0,0,0,1
3,-1.315068,0.051852,0.000000,1.317798,0.045889,0.000000,0.091843,-0.037533,0.000000,0.075944,0.463375,0.018845,0.192304,0.013745,-1.001744,0.092321,1,1,1,0,0,0,1
4,-0.068519,-0.440741,1.223679,0.156984,-0.472275,-3.170124,0.091843,-1.413414,0.000000,0.065062,0.463375,0.029073,0.097106,0.073124,0.497909,0.632632,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,-1.241742,-0.150000,1.409856,1.245247,-0.162524,-2.651452,0.497548,0.396110,0.693147,0.199081,0.093731,0.007646,0.004915,0.014207,1.497677,0.092321,1,1,1,1,0,0,1
459216,0.884725,-0.009259,1.383011,-0.858729,-0.017208,-2.307054,0.497548,0.171258,1.945910,0.199081,0.463375,0.064603,0.192304,0.039561,-1.001744,0.092321,1,1,1,1,0,0,1
459217,1.031378,-0.292593,1.361083,-1.003831,-0.309751,0.000000,0.497548,-0.037533,0.000000,0.023039,0.236296,0.020729,0.267488,0.083779,0.997793,0.092321,1,1,1,0,0,0,0
459218,-1.241742,0.642593,0.000000,1.172696,0.655832,0.000000,0.497548,-0.037533,0.000000,0.053783,0.463375,0.055488,0.267488,0.032729,-1.001744,-0.447990,1,1,1,0,0,0,1


In [23]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Convert to XGBoost's DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Enhanced parameters for multiclass classification
params = (
        objective='multi:softmax',
        random_state=42,
        learning_rate=0.05,
        max_depth=6,
        min_child_weight=8,
        subsample=0.6,
        colsample_bytree=0.85,
        gamma=0.5,
        n_estimators=450,
        reg_alpha=0.1,
        reg_lambda=0.1
    )

model = xgb.train(
    params,
    dtrain,
    num_boost_round=300,                     # Increased rounds for slower, detailed learning
    evals=[(dtrain, 'train'), (dtest, 'validation')],
    early_stopping_rounds=30,                # Increased for more stability during training
    verbose_eval=50                          # Less frequent logs for cleaner output
)

# Rest of the code remains the same
y_pred_train = model.predict(dtrain)
y_pred_test = model.predict(dtest)

train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

importance = model.get_score(importance_type='gain')
importance_df = pd.DataFrame.from_dict(importance, orient='index', columns=['importance'])
importance_df = importance_df.sort_values('importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(importance_df)

[0]	train-mlogloss:1.95850	train-merror:0.21633	validation-mlogloss:1.95852	validation-merror:0.21584
[50]	train-mlogloss:0.69435	train-merror:0.20227	validation-mlogloss:0.69884	validation-merror:0.20401
[100]	train-mlogloss:0.57860	train-merror:0.19710	validation-mlogloss:0.58679	validation-merror:0.19926
[150]	train-mlogloss:0.54723	train-merror:0.19370	validation-mlogloss:0.55938	validation-merror:0.19638
[200]	train-mlogloss:0.53241	train-merror:0.19117	validation-mlogloss:0.54884	validation-merror:0.19488
[250]	train-mlogloss:0.52265	train-merror:0.18887	validation-mlogloss:0.54365	validation-merror:0.19395
[299]	train-mlogloss:0.51531	train-merror:0.18698	validation-mlogloss:0.54076	validation-merror:0.19343
Training Metrics:
Accuracy Score: 0.8130
Macro F1 Score: 0.5193

Testing Metrics:
Accuracy Score: 0.8066
Macro F1 Score: 0.4483

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.76      0.46      0.57   

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize Logistic Regression classifier
model = LogisticRegression(
    random_state=42,
    max_iter=1000,  # Added max iterations to avoid convergence warnings
    multi_class='auto'  # Automatically choose between 'ovr' and 'multinomial'
)

# Train model
model.fit(X_train, y_train)

# Make predictions for both train and test sets
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

Training Metrics:
Accuracy Score: 0.7379
Macro F1 Score: 0.2590

Testing Metrics:
Accuracy Score: 0.7384
Macro F1 Score: 0.2591

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.29      0.00      0.00      2495
           1       0.82      0.98      0.89     58216
           2       0.32      0.05      0.08     13781
           3       0.67      0.80      0.73     29701
           4       0.42      0.32      0.37      9656
           5       0.00      0.00      0.00       842
           6       0.00      0.00      0.00        20
           7       0.00      0.00      0.00        94

    accuracy                           0.74    114805
   macro avg       0.31      0.27      0.26    114805
weighted avg       0.67      0.74      0.68    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): -0.0005
F1 Score Difference (Train - Test): -0.0001


In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize KNN classifier with basic parameters
model = KNeighborsClassifier(
    n_neighbors=5,  # Default value, can be tuned
    weights='uniform',
    metric='minkowski',
    p=2  # Euclidean distance
)

# Train model
model.fit(X_train, y_train)

# Make predictions for both train and test sets
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

# Calculate performance difference between train and test
print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

# Model specific information
print("\nModel Information:")
print(f"Number of Neighbors Used: {model.n_neighbors}")
print(f"Distance Metric: {model.metric}")
print(f"Number of Classes: {len(np.unique(y_test))}")

Training Metrics:
Accuracy Score: 0.7852
Macro F1 Score: 0.3864

Testing Metrics:
Accuracy Score: 0.7283
Macro F1 Score: 0.3078

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.50      0.21      0.30      2495
           1       0.80      0.97      0.88     58216
           2       0.23      0.08      0.12     13781
           3       0.69      0.74      0.71     29701
           4       0.54      0.39      0.45      9656
           5       0.06      0.00      0.00       842
           6       0.00      0.00      0.00        20
           7       0.00      0.00      0.00        94

    accuracy                           0.73    114805
   macro avg       0.35      0.30      0.31    114805
weighted avg       0.67      0.73      0.69    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): 0.0569
F1 Score Difference (Train - Test): 0.0786

Model Information:
Number of Neighbors Used: 5
Dist

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize Random Forest classifier with basic parameters
model = RandomForestClassifier(
    n_estimators=1000,          # More trees for better performance
    max_depth=30,               # Limit tree depth to avoid overfitting
    min_samples_split=10,       # Minimum samples required to split a node
    min_samples_leaf=5,         # Minimum samples required to be at a leaf node
    max_features='sqrt',        # Use the square root of features for splits
    random_state=42,            # Ensure reproducibility
    n_jobs=-1,                  # Use all available cores for faster computation
    bootstrap=True,             # Use bootstrap sampling to introduce randomness
    oob_score=True,             # Enable out-of-bag estimation for validation
    class_weight='balanced'     # Handle imbalanced classes if needed
)
# Train model
model.fit(X_train, y_train)

# Make predictions for both train and test sets
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

# Calculate performance difference between train and test
print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

# Feature importance (top 10)
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
})
importance_df = importance_df.sort_values('importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(importance_df)

# Model specific information
print("\nModel Information:")
print(f"Number of Trees: {model.n_estimators}")
print(f"Max Depth: {model.max_depth}")
print(f"Number of Features Used: {model.n_features_in_}")

Training Metrics:
Accuracy Score: 0.9171
Macro F1 Score: 0.9008

Testing Metrics:
Accuracy Score: 0.7943
Macro F1 Score: 0.4599

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.62      0.61      0.61      2495
           1       0.87      0.95      0.91     58216
           2       0.41      0.15      0.22     13781
           3       0.79      0.84      0.81     29701
           4       0.61      0.74      0.67      9656
           5       0.20      0.04      0.07       842
           6       0.00      0.00      0.00        20
           7       0.43      0.36      0.39        94

    accuracy                           0.79    114805
   macro avg       0.49      0.46      0.46    114805
weighted avg       0.76      0.79      0.77    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): 0.1228
F1 Score Difference (Train - Test): 0.4409

Top 10 Most Important Features:
                   